In [1]:
using Unitful
using CairoMakie
using DelimitedFiles
using PrettyTables  # Make sure this is version 3.x
import PhysicalConstants.CODATA2018: h, k_B, c_0, m_e

# Set some plot defaults
custom_theme = Theme(
    Axis = (
        xgridvisible = false,  # Set grid visibility to false
        ygridvisible = false,
    ),
)
Makie.set_theme!(custom_theme)

In [2]:
struct AtomicModel
    nlevels::Int64
    nstages::Int64
    χ::Matrix{<: Unitful.Quantity}
    g::Matrix{<: Real}
    χion::Vector{<: Unitful.Quantity}
end


"""
Reads atom structure from text file.

Parameters
----------
filename: string
    Name of file with atomic data.
"""
function read_atom(filename)
    tmp = readdlm(filename, skipstart=1)
    # Get maximum number of levels in any stage
    n_stages = Int(maximum(tmp[:, 3])) + 1
    max_levels = 0
    for i in 1:n_stages
        max_levels = max(max_levels, sum(tmp[:, 3] .== i-1))
    end
    # Populate level energies and statistical weights
    # Use a square array filled with NaNs for non-existing levels
    χ = zeros(Float64, max_levels, n_stages)
    g = zeros(Int64, max_levels, n_stages)
    for i in 1:n_stages
        nlevels = sum(tmp[: ,3] .== i-1)
        χ[1:nlevels, i] = tmp[:, 1][tmp[:, 3] .== i-1]
        g[1:nlevels, i] = tmp[:, 2][tmp[:, 3] .== i-1]
    end
    # Put units, convert from cm-1 to Joule
    χ =  (χ * u"1/cm" * h * c_0) .|> u"aJ"
    # Save ionisation energies, saved as energy of first level in each stage
    χion = copy(χ[1, :])
    # Save level energies relative to ground level in each stage
    χ .-= χion'
    χ[χ .< 0u"aJ"] .= 0u"aJ"
    return AtomicModel(max_levels, n_stages, χ, g, χion)
end

read_atom

\begin{equation} 
   U_r \equiv \sum_s g_{r,s} \mathrm{e}^{-\chi_{r,s}/kT},
\end{equation}

\begin{equation} 
    \frac{n_{r,s}}{N_r} = \frac{g_{r,s}}{U_r} \mathrm{e}^{-\chi_{r,s}/kT},
\end{equation}

\begin{equation}
   \frac{N_{r+1}}{N_r}
    = \frac{1}{N_e} \frac{2U_{r+1}}{U_r}
      \left(\frac{2 \pi m_e kT}{h^2}\right)^{3/2} 
      \mathrm{e}^{-\chi_r/kT}.
\end{equation}

1. Write code to populate the function `compute_partition_function()`
2. Write code to populate the function `compute_excitation()`
3. Write code to populate the function `compute_ionisation()`
4. Write code to populate the function `compute_populations()`
5. Write code to populate the function `plot_payne()`

In [38]:
"""
Computes partition functions using the atomic level energies and
statistical weights.

Parameters
----------
atom: AtomicModel
    Model to use.
temp: Unitful.Quantity (scalar)
    Gas temperature in units of K or equivalent.
"""
function compute_partition_function(atom, temp)
    return sum(atom.g .* exp.(-atom.χ ./ (k_B * temp)), dims=1)
end


"""
Computes the level populations relative to the ground state,
according to the Boltzmann law.

Parameters
----------
atom: AtomicModel
    Model to use.
temp:Unitful.Quantity (scalar)
    Gas temperature in units of K or equivalent.
"""
function compute_excitation(atom, temp)
    pfunc = compute_partition_function(atom, temp)
    return atom.g ./ pfunc .* exp.(-atom.χ ./ (k_B * temp))
end


"""
Computes ionisation fractions according to the Saha law.

Parameters
----------
atom: AtomicModel
    Model to use.
temp: Unitful.Quantity (scalar)
    Gas temperature in units of K or equivalent.
electron_pressure: Unitful.Quantity (scalar)
    Electron pressure in units of Pa or equivalent.
"""
function compute_ionisation(atom, temp, electron_pressure)
    pfunc = compute_partition_function(atom, temp)
    electron_density = electron_pressure / (k_B * temp)
    saha_const = (2π * m_e * k_B * temp / h^2)^(3/2)
    nstage = zeros(Float64, atom.nstages)u"m^-3"
    nstage[1] = 1.0u"m^-3"
    for r in 1:atom.nstages-1
        nstage[r + 1] = (
            nstage[r] / electron_density * 2 * pfunc[r+1] / pfunc[r] *
            saha_const * exp(-atom.χion[r+1] / (k_B * temp))
        )
    end
    return nstage / sum(nstage)
end


"""
Computes relative level populations for all levels and all
ionisation stages using the Bolzmann and Saha laws.

Parameters
----------
atom: AtomicModel
    Model to use.
temp: Unitful.Quantity (scalar)
    Gas temperature in units of K or equivalent.
electron_pressure: Unitful.Quantity (scalar)
    Electron pressure in units of Pa or equivalent.
"""
function compute_populations(atom, temp, electron_pressure)
    # Your code here
end


"""
Plots the Payne curves for the current atom.

Parameters
----------
atom: AtomicModel
    Model to use.
temperature: Unitful.Quantity (array)
    Gas temperature in units of K or equivalent.
electron_pressure: astropy.units.quantity (scalar)
    Electron pressure in units of Pa or equivalent.
"""
function plot_payne(atom, temp, electron_pressure)
    # Your code here
end

plot_payne